In [1]:
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.schema import Document
from langchain_chroma import Chroma
from dotenv import load_dotenv
from textwrap import dedent
import json
import os

load_dotenv()

True

In [2]:
MODEL_NAME = "gpt-4o-mini"
llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0)
parser = StrOutputParser()

In [169]:
with open("../../data/Raw_DB/total.json", "r", encoding="utf-8") as f:
    total = json.load(f)

contents = []
for data in total:
    if (
        data["genre"] == "로맨스"
        and "로판" not in data["keywords"]
        and "판타지" not in data["keywords"]
        and "로맨스판타지" not in data["keywords"]
        and "로맨스 판타지" not in data["keywords"]
    ):
        contents.append(data)


def formatted_contents(contents):
    """json 파일을 context로 넘길 때 사용하는 함수"""
    formatted = []
    for con in contents:
        title = con["title"]
        type = con["type"]
        platform = con["platform"]
        genre = con["genre"]
        keyword = con["keywords"]
        description = con["description"]
        status = con["status"]
        age_rating = con["age_rating"]
        episode = con["episode"]
        price = con["price_type"]
        url = con["url"]
        formatted.append(
            f"title: {title}, type: {type}, platform: {platform}, genre: {genre}, keyword: {keyword}, description: {description}, status: {status}, age_rating: {age_rating}, price: {price}, episode: {episode}, url: {url}"
        )
    return formatted


def make_Document(contents):
    """json파일을 embedding vector로 만들 때 사용하는 함수"""
    document = []
    for con in contents:
        if isinstance(con["keywords"], str):
            keyword = con["keywords"]
        else:
            keyword = ", ".join(con["keywords"])
        title = con["title"]
        type = con["type"]
        platform = con["platform"]
        description = con["description"]
        status = con["status"]
        age_rating = con["age_rating"]
        episode = con["episode"]
        price = con["price_type"]
        views = con["views"]
        rating = con["rating"]
        like = con["like"]
        author = con["author"]
        illustrator = con["illustrator"]
        original = con["original"]
        url = con["url"]
        formatted = f"title: {title}, type: {type}, platform: {platform}, keyword: {keyword}, description: {description}, status: {status}, age_rating: {age_rating}, price: {price}, episode: {episode}, views: {views}, rating: {rating}, like: {like}, author: {author}, illustrator: {illustrator}, original: {original}, url: {url}"
        document.append(
            Document(
                page_content=formatted,
                metadata={
                    "id": con["id"],
                    "title": title,
                    "type": type,
                    "platform": platform,
                    "status": status,
                },
            )
        )
    return document

In [ ]:
# vector store 생성
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

PERSIST_DIRECTORY = r"..\..\..\data\Raw_DB\vector_store\romance_only"
COLLECTION_NAME = "romance_only"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"

embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
document = make_Document(contents)

vector_store = Chroma.from_documents(
    persist_directory=PERSIST_DIRECTORY,
    collection_name=COLLECTION_NAME,
    embedding=embedding_model,
    documents=document,
)

In [190]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


@tool
def search_contents(query):
    """vector store에서 작품을 검색하는 tool"""
    PERSIST_DIRECTORY = r"..\..\..\data\Raw_DB\vector_store\romance_only"
    COLLECTION_NAME = "romance_only"
    EMBEDDING_MODEL_NAME = "text-embedding-3-small"

    embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
    vector_store = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        collection_name=COLLECTION_NAME,
        embedding_function=embedding_model,
    )
    retriever = vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 5,
            "fetch_k": 10,
            "lambda_mult": 0.2,
        },
    )
    results_retriever = retriever.invoke(query)
    return (
        results_retriever
        if results_retriever
        else [Document(page_content="검색 결과가 없습니다.")]
    )

In [206]:
from unittest import result


query = "인기많은 웹소설을 추천해줘"


def similarity_search(query, parameter):
    PERSIST_DIRECTORY = r"..\..\..\data\Raw_DB\vector_store\romance_only"
    COLLECTION_NAME = "romance_only"
    EMBEDDING_MODEL_NAME = "text-embedding-3-small"

    embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
    vector_store = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        collection_name=COLLECTION_NAME,
        embedding_function=embedding_model,
    )
    retriever = vector_store.similarity_search(query=query, filter=parameter)
    return [page.page_content for page in retriever]


def type_chain(question):
    type_genre_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "ai",
                dedent(
                    """
                    <role>
                    당신은 로맨스 웹툰과 웹소설에 대한 전문가입니다.
                    사용자의 질문에서 작품 type, platform, status, price, age_rating, keywords을 추출하십시오.
                    해당하는 카테고리가 없다면 빈칸으로 반환하십시오.
                    </role>
                    
                    <return>
                    반환 형식은 반드시 JSON 형식이어야 합니다:
                    {{
                        "type": "웹툰" 혹은 "웹소설" / 없다면 생략,
                        "status": "연재중", "완결", "금요일 연재" 등의 연재 상태 / 없다면 생략,
                        "age_rating": "19금", "청소년 이용불가", "성인" 등의 연령 제한 / 없다면 생략,
                        "price": "무료", "기다무", "유료" 등의 가격 정보 / 없다면 생략
                    }}
                    </return>
                    """
                ),
            ),
            ("human", "{question}"),
        ]
    )

    type_genre_chain = type_genre_prompt | llm | parser
    parameter = type_genre_chain.invoke(question)
    return question, parameter


question, parameter = type_chain(query)
print(question)
parameter = json.loads(parameter)
print(parameter)
result = similarity_search(question, parameter)
print(result)

인기많은 웹소설을 추천해줘
{'type': '웹소설'}
['title: X의 사정, type: 웹소설, platform: 카카오페이지, keyword: 현대로맨스, 오해물, 재회물, 첫사랑, 계약관계, 계략남, 직진남, 순정남, 대형견남, 상처남, 능력남, 절륜남, 능력녀, 상처녀, 친구>연인, description: 온 세상이 너를 버리라 한다. 독종, 술집 작부의 딸, 돈에 미친 꽃뱀. 놓고 싶지 않았다. 아무 곳에도 손 내밀 데 없는 너를. 미치도록 갖고 싶었다. 아무것에도 꺾이지 않는 너를. 차라리 더럽혀서라도. 세상 전부를 등질지라도.  ***  스타트 업 대표 신이제는 대기업과의 M&A를 앞두고 헤어진 여자친구와 재회한다. 수임 의뢰를 받고 찾아온 노무법인 ‘더 온’의 임하라와. 그런데 제 속을 갈기갈기 찢어놓고 사라진 것도 모자라 아무렇지 않은 얼굴로 찾아와 속을 뒤집는 그녀였다. “너무 오래전 일이라 기억도 안 나.”  “지금 잘 살고 있으면 된 거잖아.” 후우, 타는 숨을 뱉어낸 신이제가 거칠게 머리를 넘겼다. 뭘 안다고. 임하라 네가 뭘 안다고 함부로 지껄여. 부서질 대로 부서져 간신히 숨만 쉬고 있는 새끼한테., status: 완결, age_rating: 15세이용가, price: 1일 기다리면 무료, episode: 74, views: 304000, rating: 9.9, like: -, author: 설리연, illustrator: -, original: -, url: https://page.kakao.com/content/65936185', 'title: 집착의 한도 [19세 완전판], type: 웹소설, platform: 카카오페이지, keyword: 현대로맨스, 후회남, 카리스마남, 나쁜남자, description: “저…. 별로 재미없는 여자예요. 저는 남자… 즐겁게 하는 법도 잘 모르고요.” “예서야…. 그딴 게 걱정이야?” 한숨처럼 그녀의 이름을 부르며 강현이 물었다. “그딴 걱정은 하지 마.” 재미없는 여자라니. 네가 얼마나

In [191]:
@tool
def recommand(question):
    """사용자의 의도가 추천인 경우 사용자의 질의에서 검색을 위한 요소를 추출한 뒤 이 정보를 바탕으로 작품을 추천합니다."""
    type_genre_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "ai",
                dedent(
                    """
                    <role>
                    당신은 로맨스 웹툰과 웹소설에 대한 전문가입니다.
                    사용자의 질문에서 작품 type, platform, status, price, age_rating, keywords을 추출하십시오.
                    해당하는 카테고리가 없다면 빈칸으로 반환하십시오.
                    카테고리에 들어가지 않은 추가 정보는 키워드로 반환하십시오.
                        - '추천'은 키워드가 아닙니다.
                        - 키워드로 들어간 단어의 유의어도 키워드로 포함시키십시오.
                            예시) 정주행 = 완결, 회차 많은
                    </role>
                    
                    <return>
                    반환 형식은 반드시 JSON 형식이어야 합니다:
                    {{
                        "type": "웹툰" 혹은 "웹소설", / 없다면 "상관없음"
                        "status": "연재중", "완결", "금요일 연재" 등의 연재 상태 / 없다면 "상관없음",
                        "age_rating": "19금", "청소년 이용불가", "성인" 등의 연령 제한 / 없다면 "상관없음",
                        "price": "무료", "기다무", "유료" 등의 가격 정보 / 없다면 "상관없음",
                        "keywords": "키워드",
                    }}
                    </return>
                    """
                ),
            ),
            ("human", "{question}"),
        ]
    )

    type_genre_chain = type_genre_prompt | llm | parser

    recommendation_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "ai",
                dedent(
                    """
                    <role name = "recommandation">
                    당신은 로맨스 웹소설, 웹툰의 전문가이며 로맨스 장르의 남자주인공입니다. 당신은 모든 로맨스 장르의 웹소설과 웹툰에 박식합니다.
                    당신은 information에 주어진 사용자의 요구사항에 맞는 작품을 context를 바탕으로 추천해야합니다.
                    당신은 반드시 당신의 캐릭터대로 말을 해야합니다. 당신의 캐릭터는 사용자의 요구사항에 해당하지 않습니다.
                    </role>

                    <information>
                    사용자가 요청한 정보는 다음과 같습니다:
                    타입: {type}
                    연재 상태: {status}
                    연령 제한: {age_rating}
                    가격: {price}
                    키워드: {keywords}

                    주어진 작품 데이터:
                    {context}
                    </information>
                    
                    <check>
                    적합한 작품인지 검증:
                    type, status, age_rating, price가 사용자의 요구에 부합하는지 검증하십시오. 모든 요소에 대해 검증을 통과한 작품만 추천합니다.
                    - type:
                        사용자가 요청한 정보가 "웹툰"이면 웹툰만 추천해야 합니다.
                        사용자가 요청한 정보가 "웹소설"이면 웹소설만 추천해야 합니다.
                        사용자가 요청한 정보가 "상관없음"이면 웹툰과 웹소설 상관없이 추천해야 합니다.
                    - status:
                        사용자가 요청한 정보가 "완결"이면 무조건 해당되는 작품 중 완결 작품만 추천해야 합니다.
                    
                    추천 개수:
                    - 최대 5개를 추천합니다.
                    - 검증에 통과한 작품이 1개 이상 5개 미만일 경우: 그대로 추천합니다.
                    - 검증에 통과한 작품이 0개일 경우: 해당하는 작품이 없다고 답변하십시오.
                    </check>
                    
                    <result>
                    반환형식:
                    줄거리와 추천 이유를 제외한 각 요소는 context에 있는 그대로를 반환하십시오.
                    줄거리는 두 줄 이내로 요약해서 반환하십시오.
                    추천 이유는 context에서 해당하는 근거를 들어 설명하십시오.
                    {{
                        "title": 제목,
                        "type": 타입,
                        "platform": 플랫폼,
                        "status": 연재 상태,
                        "keyword": 키워드,
                        "description": 줄거리 요약,
                        "reason": 추천 이유,
                        "url": url
                    }}
                    </result>
                    """
                ),
            ),
        ]
    )
    recommendation_chain = recommendation_prompt | llm | parser
    complete_chain = (
        type_genre_chain
        | (
            lambda result: json.loads(result) if isinstance(result, str) else result
        )  # JSON 파싱
        | (
            lambda parsed_result: {
                "type": parsed_result["type"],
                "status": parsed_result["status"],
                "age_rating": parsed_result["age_rating"],
                "price": parsed_result["price"],
                "keywords": parsed_result["keywords"],
                "context": [
                    content.page_content for content in search_contents(question)
                ],  # 작품 데이터 JSON 전달
            }
        )
        | recommendation_chain
    )
    return complete_chain.invoke({"question": question})

In [187]:
query = "인기 많은 웹소설 추천해줘."

In [193]:
print(recommand(query))

{
    "title": "호러와 로맨스",
    "type": "웹툰",
    "platform": "네이버 웹툰",
    "status": "75화 완결",
    "keyword": "로맨스, 완결로맨스",
    "description": "사랑을 알고 싶은 호러 작가와 공포물은 질색인 인기 로맨스 작가의 오싹달콤한 로맨스 제작기.",
    "reason": "이 작품은 완결된 로맨스 웹툰으로, 독특한 설정과 매력적인 캐릭터들이 돋보입니다. 호러와 로맨스의 조화가 흥미로워 많은 독자들에게 사랑받고 있습니다.",
    "url": "https://comic.naver.com/webtoon/list?titleId=710748"
},
{
    "title": "당신의 여자가 되고 싶어요",
    "type": "웹툰",
    "platform": "네이버 웹툰",
    "status": "54화 완결",
    "keyword": "소설원작, 로맨스, 완결로맨스",
    "description": "성하윤 마음 속 갖고싶은 남자 1위, 문신휘! 다정하지만 연애에 있어서 만큼은 철벽남 신휘를 얻기 위한 계략녀 하윤의 맹랑 발칙한 계획들이 펼쳐진다.",
    "reason": "이 작품은 완결된 로맨스 웹툰으로, 주인공의 발칙한 계획과 로맨스가 흥미롭게 전개되어 많은 독자들에게 인기를 끌었습니다.",
    "url": "https://comic.naver.com/webtoon/list?titleId=780266"
}


In [188]:
intent_prompt = ChatPromptTemplate.from_messages(
    [
        # MessagesPlaceholder("agent_scratchpad"),
        (
            "ai",
            dedent(
                """
            <role>
            당신은 로맨스 웹툰과 웹소설에 대한 전문가이며 로맨스 장르의 남자주인공이기도 합니다. 사용자를 여자주인공이라 생각하고 대하십시오.
            사용자는 세 가지 의도를 가지고 있습니다.("추천", "피드백", "일상 대화")
            각 의도에 따른 행동은 아래를 참고하십시오.
            </role>

            <charactor>
            1. 당신은 사회적, 경제적으로 성공한 권위적인 인물입니다.
                - 당신은 당신 외의 모든 사람에게 지위가 낮은 사람에게 건네는 존댓말을 합니다.
                - 전혀 친절하지 않습니다. 오히려 싸가지가 없습니다
                - 화려한 사교 생활 대신 고독을 즐기며, 업무와 비즈니스 외에는 대부분의 인간관계를 멀리하는 성격입니다.  
            2. 당신의 외모는 압도적입니다. 키가 크고 수려한 외모로 사람들의 시선을 끌며, 특유의 냉철하고 날카로운 분위기로 인해 가까이 다가가기 어렵게 느껴집니다.
            3. 성격은 차갑고 이성적이며 인간미가 적습니다. 그러나 자신의 방식대로 애정을 표현하는 캐릭터입니다.  
                - 나쁜 남자의 기질을 지니며 직설적이고 까칠한 성향이 두드러집니다.  
                - 상대방에게 감정을 잘 드러내지 않지만 자신이 중요한 사람에게는 집착적인 태도를 보입니다.  
                - 사용자와의 대화에서 상황에 따라 무심하거나 직설적으로 표현합니다.  
            4. 당신의 말투는 권위적인 존댓말을 기본으로 하지만, 감정이 섞일 때는 반말과 존댓말이 섞여 나옵니다.
                - 상대방에게 몰아붙이는 듯한 대화 방식과 기대에 부응할 것을 요구하는 어조가 특징입니다.  
                - 자신의 소유욕과 불만을 서슴없이 드러냅니다.  
            5. 모든 답변은 반드시 당신의 캐릭터를 바탕으로 구성해야 합니다. 캐릭터에 벗어난 응답은 하지 마십시오.
            </charactor>
            
            <example>
            "○○씨, 대답."
            "왜 이렇게 늦게 왔어요. 나 계속 기다렸잖아."
            "내가 원하는 답이 그런 게 아니라는 건 ○○씨가 제일 잘 알잖아요."
            "대답해요. 사람 미치게 하지 말고."
            "○○씨는 참 재주가 많아요. 그중 제일 탁월한 건 사람 미치게 만드는 거?"
            "○○씨, 나 기다리게 하는 건 취미야?"
            "○○씨, 이건 꼭 알아둬요. 내가 원하는 대답은 딱 하나뿐이니까."
            </example>
            
            <recommandation>
            사용자의 의도가 "추천"인 경우, search_result를 캐릭터에 맞게 변환하여 전달하십시오.
            추천작 목록:
            {search_result}
            </recommandation>
            
            <feedback>
            사용자의 의도가 "피드백"인 경우, 따로 로그를 남기십시오.
            해당 로그의 내용은 다음을 포함해야 합니다.
            {{
                "추천작 제목": 사용자가 피드백한 제목,
                "피드백": 추천에 대한 피드백("만족", "불만족")
            }}
            </feedback>
            
            <talk>
            사용자의 의도가 "일상 대화"인 경우, 당신의 캐릭터에 맞게 일상 대화를 하십시오.
            </talk>
            """
            ),
        ),
        ("human", "{question}"),
    ]
)
intent_chain = intent_prompt | llm | parser
print(intent_chain.invoke({"question": query, "search_result": recommand(query)}))

c:\Users\USER\Documents\environments\final\Lib\site-packages\langchain_core\vectorstores\base.py:1076: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='f621e77e-2e59-4513-9650-946fa89b78d2', metadata={'id': 2334, 'platform': '카카오웹툰', 'status': '수 연재', 'title': '인소의 법칙', 'type': '웹툰'}, page_content='title: 인소의 법칙, type: 웹툰, platform: 카카오웹툰, keyword: 발랄한, 몰입되는, 로맨스, 차원이동물, description: 인터넷 소설 읽기가 취미였던 평범한 학생 함단이.\n 어느 날 자고 일어나니 옆집에는 아주 예쁜 여학생 반여령이,\n 학교에는 그 이름도 무시무시한 사대천왕이 나타났다!\n \n 하루아침에 소설처럼 뒤바뀐 세계,\n 그리고 함단이의 역할은 다름 아닌 여주인공 소꿉친구 중 하나.\n \n [인터넷 소설의 법칙!]\n 여주 친구는 각종 위험에 휩싸인다!, status: 수 연재, age_rating: 전체이용가, price: 1일마다 무료, episode: 238, views: 22293000, rating: -, like: 1038000, author: 아현, illustrator: 아현, original: 유한려, url: https://webtoon.kakao.com/content/%EC%9D%B8%EC%86%8C%EC%9D%98-%EB%B2%95%EC%B9%99/2334'), 0.10499796421171659), (Document(id='2ba53017-212b-467a-b4f2-5ace129f84f5', metadata={'id': 824888, 'platform': '네이버 웹툰', 'status': '수요웹툰', 'title'

검색 결과가 없습니다. 추천할 수 있는 작품이 없습니다. 다른 요청이 있으시면 말씀해 주세요!


In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

PERSIST_DIRECTORY = r"..\..\..\data\Raw_DB\vector_store\contents"
COLLECTION_NAME = "contents"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

vector_store = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    collection_name=COLLECTION_NAME,
    embedding_function=embedding_model,
)

retriever_1 = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
    },
)
retriever_3 = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 5,
        "score_threshold": 0.7,
    },
)

In [9]:
print("VectorStore 유사도 검색")
query = "웹툰, 로맨스"
results_similarity = vector_store.similarity_search(query)
print(
    results_similarity[0].metadata["title"] if results_similarity else "검색 결과 없음"
)

print("Retreiver 검색")

results_retreiver_1 = retriever_1.invoke(query)
results_retreiver_3 = retriever_3.invoke(query)
idx = 1
print("id\t제목\t장르\t타입\t플랫폼")
for result in results_retreiver_1:
    print(
        f"{idx}\t{result.metadata["title"]}\t{result.metadata["genre"]}\t{result.metadata["type"]}\t{result.metadata["platform"]}"
    )
    idx += 1
idx = 1
print("id\t제목\t장르\t타입\t플랫폼")
for result in results_retreiver_3:
    print(
        f"{idx}\t{result.metadata["title"]}\t{result.metadata["genre"]}\t{result.metadata["type"]}\t{result.metadata["platform"]}"
    )
    idx += 1

VectorStore 유사도 검색
호러와 로맨스
Retreiver 검색
id	제목	장르	타입	플랫폼
1	호러와 로맨스	로맨스	웹툰	네이버 웹툰
2	열애의 품격	로맨스	웹툰	카카오페이지
3	사랑도 귀농이 되나요?	로맨스	웹툰	카카오웹툰
4	오무라이스 잼잼	코믹/일상	웹툰	카카오웹툰
5	무사만리행	무협/사극	웹툰	네이버 웹툰
id	제목	장르	타입	플랫폼
1	호러와 로맨스	로맨스	웹툰	네이버 웹툰
2	열애의 품격	로맨스	웹툰	카카오페이지
3	사랑도 귀농이 되나요?	로맨스	웹툰	카카오웹툰
4	오무라이스 잼잼	코믹/일상	웹툰	카카오웹툰
5	무사만리행	무협/사극	웹툰	네이버 웹툰
